In [ ]:
from clickhouse_driver import Client

In [ ]:
client = Client(
    host='localhost',
    port=9000,
    database='fleet',
    user='fleet',
    password='fleet',
    settings={'use_numpy': True},
)

In [ ]:
longitude, latitude = client.execute(
    "select end_longitude as longitude, end_latitude as latitude "
    "from get_clean_flights(start_time=now() - interval '1 day', end_time=now()) "
    "where end_on_ground = true",
    columnar=True,
)

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
landings = pd.DataFrame({'longitude': longitude, 'latitude': latitude})
landings = landings.astype({'longitude': np.float64, 'latitude': np.float64})

In [ ]:
landings.head()

In [ ]:
import folium

In [ ]:
map = folium.Map(control_scale=True)
landings.sample(100).apply(
    lambda row: folium.Marker([row['latitude'], row['longitude']]).add_to(map),
    axis=1
)
map

In [ ]:
import cartopy as cp
from sklearn.cluster import DBSCAN

In [ ]:
projection = cp.crs.Robinson()
geodetic = cp.crs.Geodetic()
projected = projection.transform_points(geodetic, landings['longitude'], landings['latitude'])
landings['x'], landings['y'] = projected.T[:2]

In [ ]:
projection.x_limits, projection.y_limits

In [ ]:
dbscan = DBSCAN(eps=5_000, min_samples=5).fit(landings[['x', 'y']])
landings['airport'] = dbscan.labels_
landings.head()

In [ ]:
airports = landings[landings['airport'] >= 0].groupby('airport').agg(
    x=('x', 'mean'),
    y=('y', 'mean'),
    count=('airport', 'size')
)
wgs84 = geodetic.transform_points(projection, airports['x'], airports['y'])
airports['longitude'], airports['latitude'] = wgs84.T[:2]
airports.head()

In [ ]:
len(airports)

In [ ]:
map = folium.Map(control_scale=True)
airports.apply(
    lambda row: folium.Marker([row['latitude'], row['longitude']]).add_to(map),
    axis=1
)
map